In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

%autosave 60

from IPython.display import display, HTML

Autosaving every 60 seconds


In [3]:
# from pyathena import connect
# dl = connect(
#     s3_staging_dir='s3://aws-athena-query-results-782969481272-us-west-2/', #primary workgroup 
#              region_name='us-west-2')
# # 's3://sofi-data-science/tboser/money-modeling/money-customer-risk'
# # s3://aws-athena-query-results-782969481272-us-west-2/ 

In [4]:
!pip install "snowflake-connector-python[pandas]" --quiet

import snowflake.connector

def get_snowflake_connection():
    ctx = snowflake.connector.connect(
        user="<username>",
        password="<password>",
        host="localhost",
        port=1444,
        account="sdm",
        warehouse="DATA_SCIENCE",
        database="DW_PRODUCTION",
        protocol="http"
    )
    return ctx


def run_query(sql):
    with get_snowflake_connection() as ctx:
        with ctx.cursor() as cs:
            cs.execute(sql)
            allthedata = cs.fetch_pandas_all()
            return allthedata
        

#### Banking_accounts

tdm_sandbox.cleansed

tdm_bank.cleansed

In [318]:
run_query("""


-- get data at a banking account level.
WITH
    primary_account_holders
    AS
    (
        SELECT
            cif.sofi_user_id_reference  AS user_id
            , dep.account_number AS business_account_number
            , cif.date_account_opened AS date_account_opened
            , dep.account_closed_date
 AS date_account_closed
            , dep.product_type AS account_type
            , cldr.description AS account_closed_reason
            , joint.sofi_user_id_reference AS joint_user_id
        FROM tdm_bank.cleansed.profile_customers cif
            JOIN tdm_bank.cleansed.profile_deposits dep
            ON cif.customer_number = dep.customer_number
            LEFT JOIN tdm_bank.cleansed.profile_utility_closeout_reason_codes cldr ON cldr.closeout_reason_code = dep.account_closeout_reason_code
                AND cldr.product_class = dep.product_class
            LEFT JOIN
            (SELECT rc.account_number
                , cif.sofi_user_id_reference
                , rc.customer_number
            FROM tdm_bank.cleansed.profile_customer_accounts_mapping rc
                JOIN tdm_bank.cleansed.profile_customers cif ON cif.customer_number = rc.customer_number
                JOIN tdm_bank.cleansed.profile_deposits dep ON dep.account_number = rc.account_number
                JOIN tdm_bank.cleansed.profile_utility_valid_relationships_by_groups
 rel ON rel.product_group = dep.product_group
                    AND rel.relationship_code  = dep.account_relationship_code
                    AND rc.role_code_for_relationship=2) joint ON joint.account_number = dep.account_number
        WHERE dep.product_type=410
    ),
    joint_account_holders
    AS
    (
        SELECT joint_user_id AS user_id,
            business_account_number,
            date_account_opened,
            date_account_closed,
            account_type,
            account_closed_reason,
            user_id AS joint_user_id
        FROM primary_account_holders
        WHERE joint_user_id IS NOT NULL
    )
SELECT *
FROM
    (                                                                                                                SELECT *
        FROM primary_account_holders
    UNION
        SELECT *
        FROM joint_account_holders) AS internal_banking_accounts
        limit 5
""").T

,0,1,2,3,4
USER_ID,16666018,15724557,14377826,14115448,17713894
BUSINESS_ACCOUNT_NUMBER,-7139430807027193546,-3504921075161141582,3507604580546648871,4521659292770945081,-5819886073609365668
DATE_ACCOUNT_OPENED,2020-08-27,2021-11-25,2019-08-13,2019-07-07,2021-01-10
DATE_ACCOUNT_CLOSED,None,None,None,None,None
ACCOUNT_TYPE,410,410,410,410,410
ACCOUNT_CLOSED_REASON,None,None,None,None,None
JOINT_USER_ID,16675352,15747651,14589504,14262179,17715285


original - postgres

In [52]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15556,database='sofi_profile_reporting',user='rarevalo@sofi.org',password=' ')

In [310]:
pd.read_sql("""
-- get data at a banking account level.
WITH primary_account_holders AS

    
    (SELECT cif.zsofiid AS user_id,
            dep.cid AS business_account_number,
            dao AS date_account_opened,
            dtc AS date_account_closed,
            dep.type AS account_type,
            cldr.des AS account_closed_reason,
            joint.zsofiid AS joint_user_id
     FROM cif
     JOIN dep ON cif.acn = dep.acn
     LEFT JOIN utblcloser cldr ON cldr.clr = dep.clr AND cldr.cls = dep.cls
     LEFT JOIN
         (SELECT rc.cid,
                 cif.zsofiid,
                 rc.acn
          FROM relcif rc
          JOIN cif ON cif.acn = rc.acn
          JOIN dep ON dep.cid = rc.cid
          JOIN utblrel2 rel ON rel.grp = dep.grp
          AND rel.key = dep.acnrelc
          AND rc.role=2) joint ON joint.cid = dep.cid
     WHERE dep.type=410 ),
     joint_account_holders AS
    (SELECT joint_user_id AS user_id,
            business_account_number,
            date_account_opened,
            date_account_closed,
            account_type,
            account_closed_reason,
            user_id AS joint_user_id
     FROM primary_account_holders
     WHERE joint_user_id IS NOT NULL )
SELECT *
FROM
    (SELECT *
     FROM primary_account_holders
     UNION SELECT *
     FROM joint_account_holders) AS internal_banking_accounts
     
limit 5 ;     

""", conn).T

,0,1,2,3,4
user_id,188,305,378,402,419
business_account_number,410000434669,410000276182,410002894815,410000091385,410018394673
date_account_opened,2019-01-17,2018-11-11,2019-10-07,2018-07-06,2021-09-01
date_account_closed,None,None,2020-09-25,2021-01-03,None
account_type,410,410,410,410,410
account_closed_reason,None,None,Customer Request,Customer Request,None
joint_user_id,None,None,None,None,None


###### CHECK

In [319]:
%%time

banking_accounts_sf  = run_query("""

-- get data at a banking account level.
WITH
    primary_account_holders
    AS
    (
        SELECT
            cif.sofi_user_id_reference  AS user_id
            , dep.account_number AS business_account_number
            , cif.date_account_opened AS date_account_opened
            , dep.account_closed_date
 AS date_account_closed
            , dep.product_type AS account_type
            , cldr.description AS account_closed_reason
            , joint.sofi_user_id_reference AS joint_user_id
        FROM tdm_bank.cleansed.profile_customers cif
            JOIN tdm_bank.cleansed.profile_deposits dep
            ON cif.customer_number = dep.customer_number
            LEFT JOIN tdm_bank.cleansed.profile_utility_closeout_reason_codes cldr ON cldr.closeout_reason_code = dep.account_closeout_reason_code
                AND cldr.product_class = dep.product_class
            LEFT JOIN
            (SELECT rc.account_number
                , cif.sofi_user_id_reference
                , rc.customer_number
            FROM tdm_bank.cleansed.profile_customer_accounts_mapping rc
                JOIN tdm_bank.cleansed.profile_customers cif ON cif.customer_number = rc.customer_number
                JOIN tdm_bank.cleansed.profile_deposits dep ON dep.account_number = rc.account_number
                JOIN tdm_bank.cleansed.profile_utility_valid_relationships_by_groups
 rel ON rel.product_group = dep.product_group
                    AND rel.relationship_code  = dep.account_relationship_code
                    AND rc.role_code_for_relationship=2) joint ON joint.account_number = dep.account_number
        WHERE dep.product_type=410
    ),
    joint_account_holders
    AS
    (
        SELECT joint_user_id AS user_id,
            business_account_number,
            date_account_opened,
            date_account_closed,
            account_type,
            account_closed_reason,
            user_id AS joint_user_id
        FROM primary_account_holders
        WHERE joint_user_id IS NOT NULL
    )
SELECT *
FROM
    (                                                                                                                SELECT *
        FROM primary_account_holders
    UNION
        SELECT *
        FROM joint_account_holders) AS internal_banking_accounts
        
    
""")

CPU times: user 3.38 s, sys: 425 ms, total: 3.81 s
Wall time: 10.1 s


In [320]:
banking_accounts_sf.shape

(1547591, 7)

In [347]:
banking_accounts_sf.describe(include='all')

,USER_ID,BUSINESS_ACCOUNT_NUMBER,DATE_ACCOUNT_OPENED,DATE_ACCOUNT_CLOSED,ACCOUNT_TYPE,ACCOUNT_CLOSED_REASON,JOINT_USER_ID
count,1.547591e+06,1547591,1547591,155309,1547591.0,130314,2.014600e+04
unique,NaN,1537518,1552,1357,NaN,16,NaN
top,NaN,8688632849802984962,2021-08-09,2019-04-05,NaN,Customer Request,NaN
freq,NaN,2,13032,9202,NaN,48658,NaN
mean,1.791254e+07,NaN,NaN,NaN,410.0,NaN,1.570608e+07
std,3.574673e+06,NaN,NaN,NaN,0.0,NaN,4.384406e+06
min,1.880000e+02,NaN,NaN,NaN,410.0,NaN,1.794000e+03
25%,1.655617e+07,NaN,NaN,NaN,410.0,NaN,1.458626e+07
50%,1.801155e+07,NaN,NaN,NaN,410.0,NaN,1.646639e+07
75%,2.017901e+07,NaN,NaN,NaN,410.0,NaN,1.803449e+07


In [349]:
describe_df(banking_accounts_sf)

----------------------------------------
Columns: 7
Rows: 1547591
Memory usage: (337.25 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |               |      MIN |         MEAN |          MAX |         MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+---------------+----------+--------------+--------------+--------------+----------+-----------+------------
 1  | JOINT_USER_ID | 1794.000 | 15706078.226 | 24605426.000 |  3701416.000 | 98.7%    |   1527445 |      20146
 2  | USER_ID       | 188.000  | 17912542.611 | 24624255.000 | 18691470.000 | 0.0%     |         0 |    1547591
 3  | ACCOUNT_TYPE  | 410.000  | 410.000      | 410.000      |      410.000 | 0.0%     |         0 |    1547591
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                         |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+-------------------------+----------+-----------+------------+-----------

In [53]:
%%time

banking_accounts_postgres = pd.read_sql("""
-- get data at a banking account level.
WITH primary_account_holders AS

    
    (SELECT cif.zsofiid AS user_id,
            dep.cid AS business_account_number,
            dao AS date_account_opened,
            dtc AS date_account_closed,
            dep.type AS account_type,
            cldr.des AS account_closed_reason,
            joint.zsofiid AS joint_user_id
     FROM cif
     JOIN dep ON cif.acn = dep.acn
     LEFT JOIN utblcloser cldr ON cldr.clr = dep.clr AND cldr.cls = dep.cls
     LEFT JOIN
         (SELECT rc.cid,
                 cif.zsofiid,
                 rc.acn
          FROM relcif rc
          JOIN cif ON cif.acn = rc.acn
          JOIN dep ON dep.cid = rc.cid
          JOIN utblrel2 rel ON rel.grp = dep.grp
          AND rel.key = dep.acnrelc
          AND rc.role=2) joint ON joint.cid = dep.cid
     WHERE dep.type=410 ),
     joint_account_holders AS
    (SELECT joint_user_id AS user_id,
            business_account_number,
            date_account_opened,
            date_account_closed,
            account_type,
            account_closed_reason,
            user_id AS joint_user_id
     FROM primary_account_holders
     WHERE joint_user_id IS NOT NULL )
SELECT *
FROM
    (SELECT *
     FROM primary_account_holders
     UNION SELECT *
     FROM joint_account_holders) AS internal_banking_accounts
     


""", conn) 

CPU times: user 2.79 s, sys: 260 ms, total: 3.05 s
Wall time: 11.1 s


In [312]:
banking_accounts_postgres.shape

(1547591, 7)

In [348]:
banking_accounts_postgres.describe(include='all')

,user_id,business_account_number,date_account_opened,date_account_closed,account_type,account_closed_reason,joint_user_id
count,1.547591e+06,1.547591e+06,1547591,155309,1547591.0,130314,2.014600e+04
unique,NaN,NaN,1552,1357,NaN,16,NaN
top,NaN,NaN,2021-08-09,2019-04-05,NaN,Customer Request,NaN
freq,NaN,NaN,13032,9202,NaN,48658,NaN
mean,1.791254e+07,4.100138e+11,NaN,NaN,410.0,NaN,1.570608e+07
std,3.574673e+06,7.439566e+06,NaN,NaN,0.0,NaN,4.384406e+06
min,1.880000e+02,4.100001e+11,NaN,NaN,410.0,NaN,1.794000e+03
25%,1.655617e+07,4.100078e+11,NaN,NaN,410.0,NaN,1.458626e+07
50%,1.801155e+07,4.100135e+11,NaN,NaN,410.0,NaN,1.646639e+07
75%,2.017901e+07,4.100200e+11,NaN,NaN,410.0,NaN,1.803449e+07


In [67]:
banking_accounts_postgres.business_account_number = banking_accounts_postgres.business_account_number.astype('category')

In [68]:
# (banking_accounts_postgres.assign(business_account_number = banking_accounts_postgres.business_account_number.astype('category')).pipe( lambda  banking_accounts_postgres :describe_df(banking_accounts_postgres)))

In [69]:
describe_df(banking_accounts_postgres)

----------------------------------------
Columns: 7
Rows: 1547921
Memory usage: (232.59 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |               |      MIN |         MEAN |          MAX |         MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+---------------+----------+--------------+--------------+--------------+----------+-----------+------------
 1  | joint_user_id | 1794.000 | 15709602.624 | 24637790.000 |  3701416.000 | 98.7%    |   1527797 |      20124
 2  | user_id       | 188.000  | 17915895.508 | 24648109.000 | 18691470.000 | 0.0%     |         0 |    1547921
 3  | account_type  | 410.000  | 410.000      | 410.000      |      410.000 | 0.0%     |         0 |    1547921
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                         |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+-------------------------+----------+-----------+------------+-----------

#### Experian credit pull

In [80]:
run_query(""" 

-- Query for Experian credit pull
SELECT b.user_id,
       cp.credit_pull_date,
       cp.fico_score,
       cp.vantage_score,
       parse_json(cp.experian_attribute_json):ALL7120 as ALL7120,
       parse_json(cp.experian_attribute_json):ALL8220 as ALL8220,
       parse_json(cp.experian_attribute_json):BCC2800 as BCC2800,
       parse_json(cp.experian_attribute_json):BCC7120 as BCC7120,
       parse_json(cp.experian_attribute_json):BCX3423 as BCX3423,
       parse_json(cp.experian_attribute_json):ILN5520 as ILN5520,
       parse_json(cp.experian_attribute_json):IQT9415 as IQT9415,
       parse_json(cp.experian_attribute_json):IQT9413 as IQT9413,
       parse_json(cp.experian_attribute_json):MTF5820 as MTF5820,
       parse_json(cp.experian_attribute_json):STU5031 as STU5031,
       cp.credit_card_loan_amount,
       cp.delinquencies_90_days,
       cp.education_loan_amount,
       cp.mortgage_loan_amount,
       cp.secured_loan_amount,
       cp.total_outstanding_balance,
       cp.total_tradelines_open,
       cp.unsecured_loan_amount
FROM TDM_RISK_MGMT_HUB.modeled.credit_pull cp
JOIN TDM_RISK_MGMT_HUB.modeled.borrowers b ON b.borrower_id = cp.borrower_id
WHERE cp.pull_type = 'MONEY'

limit 5""").T

,0,1,2,3,4
USER_ID,13142716,22441203,22430619,22430619,22430619
CREDIT_PULL_DATE,2019-01-26,2022-02-15,2022-02-14,2022-02-14,2022-02-14
FICO_SCORE,415,426,NaN,NaN,NaN
VANTAGE_SCORE,483,455,NaN,NaN,NaN
ALL7120,"""996""","""156""",None,None,None
ALL8220,"""51""","""79""",None,None,None
BCC2800,"""0""","""1""",None,None,None
BCC7120,"""996""","""156""",None,None,None
BCX3423,"""0""","""1""",None,None,None
ILN5520,"""999999997""","""8371""",None,None,None


postgres

In [73]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15501,database='sofi_dw',user='rarevalo@sofi.org',password=' ')

In [31]:
pd.read_sql("""-- Query for Experian credit pull
SELECT b.user_id,
       cp.credit_pull_date,
       cp.fico_score,
       cp.vantage_score,
       cp.experian_attribute_json::json->>'ALL7120' AS ALL7120,
       cp.experian_attribute_json::json->>'ALL8220' AS ALL8220,
       cp.experian_attribute_json::json->>'BCC2800' AS BCC2800,
       cp.experian_attribute_json::json->>'BCC7120' AS BCC7120,
       cp.experian_attribute_json::json->>'BCX3423' AS BCX3423,
       cp.experian_attribute_json::json->>'ILN5520' AS ILN5520,
       cp.experian_attribute_json::json->>'IQT9415' AS IQT9415,
       cp.experian_attribute_json::json->>'IQT9413' AS IQT9413,
       cp.experian_attribute_json::json->>'MTF5820' AS MTF5820,
       cp.experian_attribute_json::json->>'STU5031' AS STU5031,
       cp.credit_card_loan_amount,
       cp.delinquencies_90_days,
       cp.education_loan_amount,
       cp.mortgage_loan_amount,
       cp.secured_loan_amount,
       cp.total_outstanding_balance,
       cp.total_tradelines_open,
       cp.unsecured_loan_amount
FROM credit_pull cp
JOIN borrowers b ON b.borrower_id = cp.borrower_id
WHERE cp.pull_type = 'MONEY'

limit 5 


""", conn).T

,0,1,2,3,4
user_id,17389,17723,19037306,19045039,19037491
credit_pull_date,2019-10-20,2019-10-20,2021-06-16,2021-06-16,2021-06-16
fico_score,789,NaN,9002,526,706
vantage_score,797,NaN,563,514,758
all7120,91,None,997,114,99
all8220,176,None,26,37,326
bcc2800,0,None,98,1,0
bcc7120,18,None,998,114,33
bcx3423,0,None,98,1,0
iln5520,999999997,None,999999997,999999998,28006


###### CHECK

In [81]:
%%time

experian_credit_pull_sf = run_query(""" 
-- Query for Experian credit pull
SELECT b.user_id,
       cp.credit_pull_date,
       cp.fico_score,
       cp.vantage_score,
       parse_json(cp.experian_attribute_json):ALL7120 as ALL7120,
       parse_json(cp.experian_attribute_json):ALL8220 as ALL8220,
       parse_json(cp.experian_attribute_json):BCC2800 as BCC2800,
       parse_json(cp.experian_attribute_json):BCC7120 as BCC7120,
       parse_json(cp.experian_attribute_json):BCX3423 as BCX3423,
       parse_json(cp.experian_attribute_json):ILN5520 as ILN5520,
       parse_json(cp.experian_attribute_json):IQT9415 as IQT9415,
       parse_json(cp.experian_attribute_json):IQT9413 as IQT9413,
       parse_json(cp.experian_attribute_json):MTF5820 as MTF5820,
       parse_json(cp.experian_attribute_json):STU5031 as STU5031,
       cp.credit_card_loan_amount,
       cp.delinquencies_90_days,
       cp.education_loan_amount,
       cp.mortgage_loan_amount,
       cp.secured_loan_amount,
       cp.total_outstanding_balance,
       cp.total_tradelines_open,
       cp.unsecured_loan_amount
FROM TDM_RISK_MGMT_HUB.modeled.credit_pull cp
JOIN TDM_RISK_MGMT_HUB.modeled.borrowers b ON b.borrower_id = cp.borrower_id
WHERE cp.pull_type = 'MONEY'

""")

CPU times: user 7.69 s, sys: 891 ms, total: 8.58 s
Wall time: 1min


In [82]:
describe_df(experian_credit_pull_sf)

----------------------------------------
Columns: 22
Rows: 3176589
Memory usage: (2114.32 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                           |     MIN |         MEAN |          MAX |         MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+---------------------------+---------+--------------+--------------+--------------+----------+-----------+------------
 1  | VANTAGE_SCORE             | 1.000   | 565.792      | 839.000      |        4.000 | 28.0%    |    889382 |    2287207
 2  | FICO_SCORE                | 378.000 | 2364.823     | 9003.000     |     9003.000 | 28.0%    |    889375 |    2287214
 3  | USER_ID                   | 188.000 | 18311061.155 | 24647698.000 | 21574548.000 | 0.0%     |         0 |    3176589
 4  | CREDIT_CARD_LOAN_AMOUNT   | 0.000   | 2527.632     | 4077537.000  |        0.000 | 0.0%     |         0 |    3176589
 5  | EDUCATION_LOAN_AMOUNT     | 0.000   | 5681.232     | 1959837.000  |        0.000 | 0.0%     |      

In [74]:
%%time

experian_credit_pull_postgres = pd.read_sql("""-- Query for Experian credit pull
SELECT b.user_id,
       cp.credit_pull_date,
       cp.fico_score,
       cp.vantage_score,
       cp.experian_attribute_json::json->>'ALL7120' AS ALL7120,
       cp.experian_attribute_json::json->>'ALL8220' AS ALL8220,
       cp.experian_attribute_json::json->>'BCC2800' AS BCC2800,
       cp.experian_attribute_json::json->>'BCC7120' AS BCC7120,
       cp.experian_attribute_json::json->>'BCX3423' AS BCX3423,
       cp.experian_attribute_json::json->>'ILN5520' AS ILN5520,
       cp.experian_attribute_json::json->>'IQT9415' AS IQT9415,
       cp.experian_attribute_json::json->>'IQT9413' AS IQT9413,
       cp.experian_attribute_json::json->>'MTF5820' AS MTF5820,
       cp.experian_attribute_json::json->>'STU5031' AS STU5031,
       cp.credit_card_loan_amount,
       cp.delinquencies_90_days,
       cp.education_loan_amount,
       cp.mortgage_loan_amount,
       cp.secured_loan_amount,
       cp.total_outstanding_balance,
       cp.total_tradelines_open,
       cp.unsecured_loan_amount
FROM credit_pull cp
JOIN borrowers b ON b.borrower_id = cp.borrower_id
WHERE cp.pull_type = 'MONEY'


""", conn)

CPU times: user 30 s, sys: 3.2 s, total: 33.2 s
Wall time: 24min 19s


In [75]:
describe_df(experian_credit_pull_postgres)

----------------------------------------
Columns: 22
Rows: 3176589
Memory usage: (1892.25 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                           |     MIN |         MEAN |          MAX |         MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+---------------------------+---------+--------------+--------------+--------------+----------+-----------+------------
 1  | vantage_score             | 1.000   | 565.792      | 839.000      |        4.000 | 28.0%    |    889382 |    2287207
 2  | fico_score                | 378.000 | 2364.823     | 9003.000     |     9003.000 | 28.0%    |    889375 |    2287214
 3  | user_id                   | 188.000 | 18311061.155 | 24647698.000 | 21574548.000 | 0.0%     |         0 |    3176589
 4  | credit_card_loan_amount   | 0.000   | 2527.632     | 4077537.000  |        0.000 | 0.0%     |         0 |    3176589
 5  | education_loan_amount     | 0.000   | 5681.232     | 1959837.000  |        0.000 | 0.0%     |      

#### Socure

In [133]:
run_query("""

-- Query for Socure data pull
with map as (select distinct thirdparty_raw_id, event_id
             from TDM_RISK_MGMT_HUB.CLEANSED.decision)
select e.user_id
       , tpr.created_dt
       , case when parse_json(tpr.response):fraud.scores[0].version = '3.0' 
        then parse_json(tpr.response):fraud.scores[0].score 
        else parse_json(tpr.response):fraud.scores[1].score end as fraud_score_1_v1

       , case when parse_json(tpr.response):fraud.scores[0].version = '1.0'  
        then parse_json(tpr.response):fraud.scores[0].score 
        else parse_json(tpr.response):fraud.scores[1].score end as fraud_score_2_v1

       , case when parse_json(tpr.response):fraud.scores[0].name='generic' 
        then parse_json(tpr.response):fraud.scores[0].score 
        when parse_json(tpr.response):fraud.scores[1].name='generic' 
        then parse_json(tpr.response):fraud.scores[1].score
        else null end as fraud_score_1

       , case when parse_json(tpr.response):fraud.scores[0].name='generic' 
        then parse_json(tpr.response):fraud.scores[0].version
        when parse_json(tpr.response):fraud.scores[1].name='generic' 
        then parse_json(tpr.response):fraud.scores[1].version
        else null end as fraud_score_1_version

       , case when parse_json(tpr.response):fraud.scores[0].name='sigma' 
        then parse_json(tpr.response):fraud.scores[0].score 
        when parse_json(tpr.response):fraud.scores[1].name='sigma' 
        then parse_json(tpr.response):fraud.scores[1].score
        else null end as fraud_score_2        
           
       , case when parse_json(tpr.response):fraud.scores[0].name='sigma' 
        then parse_json(tpr.response):fraud.scores[0].version
        when parse_json(tpr.response):fraud.scores[1].name='sigma' 
        then parse_json(tpr.response):fraud.scores[1].version
        else null end as fraud_score_2_version

       , parse_json(tpr.response):addressRisk.score as address_risk_score
       , parse_json(tpr.response):emailRisk.score as email_risk_score
       , parse_json(tpr.response):phoneRisk.score as phone_risk_score
       , parse_json(tpr.response):nameAddressCorrelation.score as name_address_correlation
       , parse_json(tpr.response):nameEmailCorrelation.score as name_email_correlation
       , parse_json(tpr.response):namePhoneCorrelation.score as name_phone_correlation
       , parse_json(tpr.response):social.profilesFound as social_profiles_found
       
from TDM_RISK.CLEANSED.event e
join map on e.event_id=map.event_id
join TDM_RISK.CLEANSED.thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
join TDM_RISK.CLEANSED.thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='SOCURE' and e.user_id in(14511527,22012089,15072976)

limit 5
""")



,USER_ID,CREATED_DT,FRAUD_SCORE_1_V1,FRAUD_SCORE_2_V1,FRAUD_SCORE_1,FRAUD_SCORE_1_VERSION,FRAUD_SCORE_2,FRAUD_SCORE_2_VERSION,ADDRESS_RISK_SCORE,EMAIL_RISK_SCORE,PHONE_RISK_SCORE,NAME_ADDRESS_CORRELATION,NAME_EMAIL_CORRELATION,NAME_PHONE_CORRELATION,SOCIAL_PROFILES_FOUND
0,14511527,2020-02-05 13:01:41.316000-08:00,0.0941,0.36,0.0941,"""3.0""",0.36,"""1.0""",0.536,0.557,0.643,0.9096,0.995,0.9984,"[\n ""https://twitter.com/First_Lady_C""\n]"
1,15072976,2019-11-23 20:21:36.262000-08:00,0.0237,0.164,0.0237,"""3.0""",0.164,"""1.0""",0.227,0.249,0.32,0.9096,0.9959,0.9984,"[\n ""https://twitter.com/laineeblack""\n]"
2,22012089,2022-01-19 12:11:55.934000-08:00,None,None,None,None,0.565,"""2.0""",0.443,0.461,0.946,0.6849,0.9999,0.01,None
3,14511527,2019-10-20 18:42:43.935000-07:00,0.0263,0.359,0.0263,"""3.0""",0.359,"""1.0""",0.411,0.631,0.516,0.9096,0.995,0.9984,"[\n ""https://twitter.com/?profile_id=34679088..."
4,15072976,2020-03-01 23:15:40.033000-08:00,0.1651,0.433,0.1651,"""3.0""",0.433,"""1.0""",0.387,0.629,0.197,0.9096,0.9959,0.9984,"[\n ""https://twitter.com/laineeblack""\n]"


Postgres

In [44]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15722,database='sofi_fraud',user='rarevalo@sofi.org',password=' ')

In [45]:
pd.read_sql("""with map as (select distinct thirdparty_raw_id, event_id
             from decision)
  
             
select e.user_id,
       tpr.created_dt,
       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'version'='3.0' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' else tpr.response::json->'fraud'#>'{scores, 1}'->>'score' end as fraud_score_1_v1,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'version'='1.0' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' else tpr.response::json->'fraud'#>'{scores, 1}'->>'score' end as fraud_score_2_v1,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 1}'->>'score'
          else null end as fraud_score_1,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 0}'->>'version' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 1}'->>'version'
          else null end as fraud_score_1_version,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 1}'->>'score'
          else null end as fraud_score_2,

          
       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 0}'->>'version' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 1}'->>'version'
          else null end as fraud_score_2_version,
       tpr.response::jsonb->'addressRisk'->>'score' as address_risk_score,        
       tpr.response::jsonb->'emailRisk'->>'score' as email_risk_score,
       tpr.response::jsonb->'phoneRisk'->>'score' as phone_risk_score,
       tpr.response::jsonb->'nameAddressCorrelation'->>'score' as name_address_correlation,
       tpr.response::jsonb->'nameEmailCorrelation'->>'score' as name_email_correlation,
       tpr.response::jsonb->'namePhoneCorrelation'->>'score' as name_phone_correlation,
       tpr.response::jsonb->'social'->>'profilesFound' as social_profiles_found
from event e
join map on e.event_id=map.event_id
join thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
join thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='SOCURE' and e.user_id in ('14511527','22012089','15072976')
limit 5 """,conn)

,user_id,created_dt,fraud_score_1_v1,fraud_score_2_v1,fraud_score_1,fraud_score_1_version,fraud_score_2,fraud_score_2_version,address_risk_score,email_risk_score,phone_risk_score,name_address_correlation,name_email_correlation,name_phone_correlation,social_profiles_found
0,22012089,2022-01-19 20:11:55.934196+00:00,None,None,None,None,0.565,2.0,0.443,0.461,0.946,0.6849,0.9999,0.01,None
1,15072976,2019-11-24 04:21:36.262997+00:00,0.0237,0.164,0.0237,3.0,0.164,1.0,0.227,0.249,0.32,0.9096,0.9959,0.9984,"[""https://twitter.com/laineeblack""]"
2,15072976,2020-03-02 07:15:40.033707+00:00,0.1651,0.433,0.1651,3.0,0.433,1.0,0.387,0.629,0.197,0.9096,0.9959,0.9984,"[""https://twitter.com/laineeblack""]"
3,14511527,2019-10-21 01:42:43.935185+00:00,0.0263,0.359,0.0263,3.0,0.359,1.0,0.411,0.631,0.516,0.9096,0.995,0.9984,"[""https://twitter.com/?profile_id=346790889"", ..."
4,14511527,2020-02-05 21:01:41.316520+00:00,0.0941,0.36,0.0941,3.0,0.36,1.0,0.536,0.557,0.643,0.9096,0.995,0.9984,"[""https://twitter.com/First_Lady_C""]"


###### CHECK

Socure_postgres

In [ ]:
%%time 
socure_postgres = pd.read_sql("""with map as (select distinct thirdparty_raw_id, event_id
             from decision)
  
             
select e.user_id,
       tpr.created_dt,
       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'version'='3.0' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' else tpr.response::json->'fraud'#>'{scores, 1}'->>'score' end as fraud_score_1_v1,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'version'='1.0' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' else tpr.response::json->'fraud'#>'{scores, 1}'->>'score' end as fraud_score_2_v1,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 1}'->>'score'
          else null end as fraud_score_1,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 0}'->>'version' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='generic' then tpr.response::json->'fraud'#>'{scores, 1}'->>'version'
          else null end as fraud_score_1_version,

       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 0}'->>'score' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 1}'->>'score'
          else null end as fraud_score_2,

          
       case when tpr.response::jsonb->'fraud'#>'{scores, 0}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 0}'->>'version' 
          when tpr.response::jsonb->'fraud'#>'{scores, 1}'->>'name'='sigma' then tpr.response::json->'fraud'#>'{scores, 1}'->>'version'
          else null end as fraud_score_2_version,
       tpr.response::jsonb->'addressRisk'->>'score' as address_risk_score,        
       tpr.response::jsonb->'emailRisk'->>'score' as email_risk_score,
       tpr.response::jsonb->'phoneRisk'->>'score' as phone_risk_score,
       tpr.response::jsonb->'nameAddressCorrelation'->>'score' as name_address_correlation,
       tpr.response::jsonb->'nameEmailCorrelation'->>'score' as name_email_correlation,
       tpr.response::jsonb->'namePhoneCorrelation'->>'score' as name_phone_correlation,
       tpr.response::jsonb->'social'->>'profilesFound' as social_profiles_found
from event e
join map on e.event_id=map.event_id
join thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
join thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='SOCURE' 
""",conn)

socure_postgres.to_csv('socure_postgres.csv')


In [13]:
from pyutils import *

In [17]:
def describe_df(df,floatfmt= '.3f'):
    # Numerical
    print("--" * 20)
    print('Columns:', df.shape[1])
    print('Rows:', df.shape[0])
    print("Memory usage:", (f"({(sys.getsizeof(df) / 1024 ** 2):.2f} Mb)"))

    print("--" * 20)
    print('NUMERICAL VARIABLES:')

    numerical = df.select_dtypes(include=np.number)
    concatenated_numerical = pd.concat([
        round(numerical.isnull().sum() / df.shape[0] * 100, 2).astype(str) + "%",
        numerical.isnull().sum(),
        numerical.count(),
        numerical.min(),
        numerical.mean(),
        numerical.max()
    ], axis=1, keys=["%NULLS", "COUNT_NULLS", "NOT_NULL", 'MIN', 'MEAN', 'MAX'], sort=False).sort_values('COUNT_NULLS',
                                                                                                         ascending=False).reset_index().rename(
        columns={'index': ''})

    t = numerical.mode().T
    t.rename(columns={0: 'MODE'}, inplace=True)
    concatenated_numerical = concatenated_numerical.merge(t, how='left', left_on='', right_on=t.index)
    concatenated_numerical.index = concatenated_numerical.index + 1
    concatenated_numerical = concatenated_numerical.iloc[:, [0, 4, 5, 6, 7, 1, 2, 3]]

    print(tabulate(concatenated_numerical, headers=
    [
        'MIN',
        'MEAN',
        'MAX',
        'MODE',
        "%NULLS",
        "#_NULLS",
        "NOT_NULL",

    ], tablefmt="presto", 
#                    colalign=("right"), 
                   floatfmt=floatfmt))

    # Categorical

    print('-----' * 20)
    print()
    print('CATEGORICAL VARIABLES:')
    categorical = df.select_dtypes(['object','category'])
    if categorical.shape[1] == 0:
        print("No Categorical Variables")
    else:
        concatenated_categorical = pd.concat([

            round(categorical.isnull().sum() / df.shape[0] * 100, 2).astype(str) + "%",
            categorical.isnull().sum(),
            categorical.count()
        ],

            keys=["%NULLS",
                  "COUNT_NULLS",
                  "NOT_NULL"], axis=1, sort=False).sort_values('%NULLS', ascending=False).reset_index().rename(
            columns={'index': ''})

        max_unique = 5
        u_strs = []

        for col in categorical:
            series = categorical.loc[categorical[col].notnull(), col]
            n_unique = series.nunique()
            if n_unique > max_unique:
                u_strs.append(str(n_unique) + ' unique values')
            else:
                u_strs.append(str(series.unique()))

        t = pd.DataFrame(u_strs, categorical.columns)
        t = t.reset_index()
        t = t.rename(columns={'index': '', 0: 'Unique_Values'})
        concatenated_categorical = concatenated_categorical.merge(t, on='')
        concatenated_categorical.index = concatenated_categorical.index + 1

        print(tabulate(concatenated_categorical, headers=
        [
            "%NULLS",
            "#_NULLS",
            "NOT_NULL",
            "Unique_Values"

        ], tablefmt="presto", colalign=("left")))


In [93]:
socure_postgres = pd.read_csv('socure_postgres.csv',  index_col=0)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [98]:
socure_postgres[['user_id','created_dt']] = socure_postgres[['user_id','created_dt']].astype('category')

In [100]:
describe_df(socure_postgres)

----------------------------------------
Columns: 15
Rows: 3872938
Memory usage: (1190.82 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                          |   MIN |   MEAN |   MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+--------------------------+-------+--------+-------+--------+----------+-----------+------------
 1  | fraud_score_2_v1         | 0.001 | 0.547  | 0.999 |  0.999 | 61.91%   |   2397747 |    1475191
 2  | fraud_score_1            | 0.008 | 0.118  | 0.998 |  0.023 | 61.13%   |   2367356 |    1505582
 3  | fraud_score_1_version    | 3.000 | 3.006  | 9.000 |  3.000 | 61.13%   |   2367356 |    1505582
 4  | fraud_score_1_v1         | 0.001 | 0.207  | 0.999 |  0.025 | 50.33%   |   1949162 |    1923776
 5  | fraud_score_2            | 0.001 | 0.524  | 0.999 |  0.999 | 2.3%     |     88982 |    3783956
 6  | fraud_score_2_version    | 1.000 | 1.858  | 3.000 |  2.000 | 2.3%     |     88982 |    3783956
 7  | address_risk_score       | 0.

Socure SF

In [137]:
%%time 
socure_sf = run_query("""

-- Query for Socure data pull
with map as (select distinct thirdparty_raw_id, event_id
             from TDM_RISK_MGMT_HUB.CLEANSED.decision)
select e.user_id
       , tpr.created_dt
       , case when parse_json(tpr.response):fraud.scores[0].version = '3.0' 
        then parse_json(tpr.response):fraud.scores[0].score 
        else parse_json(tpr.response):fraud.scores[1].score end as fraud_score_1_v1

       , case when parse_json(tpr.response):fraud.scores[0].version = '1.0'  
        then parse_json(tpr.response):fraud.scores[0].score 
        else parse_json(tpr.response):fraud.scores[1].score end as fraud_score_2_v1

       , case when parse_json(tpr.response):fraud.scores[0].name='generic' 
        then parse_json(tpr.response):fraud.scores[0].score 
        when parse_json(tpr.response):fraud.scores[1].name='generic' 
        then parse_json(tpr.response):fraud.scores[1].score
        else null end as fraud_score_1

       , case when parse_json(tpr.response):fraud.scores[0].name='generic' 
        then parse_json(tpr.response):fraud.scores[0].version
        when parse_json(tpr.response):fraud.scores[1].name='generic' 
        then parse_json(tpr.response):fraud.scores[1].version
        else null end as fraud_score_1_version

       , case when parse_json(tpr.response):fraud.scores[0].name='sigma' 
        then parse_json(tpr.response):fraud.scores[0].score 
        when parse_json(tpr.response):fraud.scores[1].name='sigma' 
        then parse_json(tpr.response):fraud.scores[1].score
        else null end as fraud_score_2        
           
       , case when parse_json(tpr.response):fraud.scores[0].name='sigma' 
        then parse_json(tpr.response):fraud.scores[0].version
        when parse_json(tpr.response):fraud.scores[1].name='sigma' 
        then parse_json(tpr.response):fraud.scores[1].version
        else null end as fraud_score_2_version

       , parse_json(tpr.response):addressRisk.score as address_risk_score
       , parse_json(tpr.response):emailRisk.score as email_risk_score
       , parse_json(tpr.response):phoneRisk.score as phone_risk_score
       , parse_json(tpr.response):nameAddressCorrelation.score as name_address_correlation
       , parse_json(tpr.response):nameEmailCorrelation.score as name_email_correlation
       , parse_json(tpr.response):namePhoneCorrelation.score as name_phone_correlation
       , parse_json(tpr.response):social.profilesFound as social_profiles_found
       
from TDM_RISK.CLEANSED.event e
join map on e.event_id=map.event_id
join TDM_RISK.CLEANSED.thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
join TDM_RISK.CLEANSED.thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='SOCURE' 

""")

CPU times: user 9.14 s, sys: 1.52 s, total: 10.7 s
Wall time: 1min 5s


In [138]:
socure_sf.shape

(3875823, 15)

In [140]:
socure_sf = socure_sf.replace('"', '', regex=True)

In [145]:
socure_sf.iloc[:,2:-1] = socure_sf.iloc[:,2:-1].astype('float64')

In [148]:
socure_sf.CREATED_DT = socure_sf.CREATED_DT.astype('object')

In [149]:
describe_df(socure_sf)

----------------------------------------
Columns: 15
Rows: 3875823
Memory usage: (1217.98 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                          |   MIN |   MEAN |   MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+--------------------------+-------+--------+-------+--------+----------+-----------+------------
 1  | FRAUD_SCORE_2_V1         | 0.001 | 0.547  | 0.999 |  0.999 | 61.94%   |   2400632 |    1475191
 2  | FRAUD_SCORE_1            | 0.008 | 0.118  | 0.998 |  0.023 | 61.15%   |   2370241 |    1505582
 3  | FRAUD_SCORE_1_VERSION    | 3.000 | 3.006  | 9.000 |  3.000 | 61.15%   |   2370241 |    1505582
 4  | FRAUD_SCORE_1_V1         | 0.001 | 0.207  | 0.999 |  0.025 | 50.29%   |   1949168 |    1926655
 5  | FRAUD_SCORE_2            | 0.001 | 0.524  | 0.999 |  0.999 | 2.3%     |     88988 |    3786835
 6  | FRAUD_SCORE_2_VERSION    | 1.000 | 1.859  | 3.000 |  2.000 | 2.3%     |     88988 |    3786835
 7  | ADDRESS_RISK_SCORE       | 0.

#### Transactions

In [ ]:

transactions_sf = run_query("""
-- banking transactions by business account.
SELECT system_processing_date
    , dtj.clock_time
    , dtj.account_number AS business_account_number
    , dtj.calendar_date AS created_dt
    , dtj.ending_balance as endbal
    , dtj.external_transaction_code AS transaction_code
    , dtj.transaction_amount AS transaction_amount
    , trn.debit_or_credit as is_credit
    , cif.sofi_user_id_reference as user_id
    , split_part(split_part(dtj.transaction_source_tags, 'RCID#', 2), '~', 1) AS external_account_number
    , split_part(split_part(dtj.transaction_source_tags, 'RINS#', 2), '~', 1) AS external_institution_id 
    , split_part(split_part(dtj.transaction_source_tags, 'ACHCOID#', 2), '~', 1) AS originating_company_id
    , split_part(split_part(dtj.transaction_source_tags, 'ACHID#', 2), '~', 1) AS external_institution_trans_id
    , split_part(split_part(dtj.transaction_source_tags, 'ACHODFI#', 2), '~', 1) AS originator_dfi_id
    , auth.merchant_name AS merchant_name
FROM tdm_bank.cleansed.profile_daily_transaction_journal dtj
    LEFT JOIN tdm_bank.cleansed.profile_authorizations auth ON auth.authorization_id = split_part(split_part(dtj.transaction_source_tags, 'AUTHID#', 2), '~', 1) AND dtj.transaction_source_tags LIKE '%%AUTHID#%%'
    JOIN tdm_bank.cleansed.profile_deposits dep ON dep.account_number = dtj.account_number
    JOIN tdm_bank.cleansed.profile_customers cif ON cif.customer_number = dep.customer_number
    LEFT JOIN tdm_bank.cleansed.profile_transaction_codes trn on dtj.external_transaction_code = trn.transaction_code
WHERE endbal IS NOT NULL
    and dtj.transaction_amount IS NOT NULL
    and dtj.transaction_amount NOT LIKE '%%#%%'

""")

In [9]:
run_query("""

-- banking transactions by business account.
SELECT system_processing_date
    , dtj.clock_time
    , dtj.account_number AS business_account_number
    , dtj.calendar_date AS created_dt
    , ending_balance as endbal
    , dtj.external_transaction_code AS transaction_code
    , dtj.transaction_amount AS transaction_amount
    , trn.debit_or_credit as is_credit
    , cif.sofi_user_id_reference as user_id
    , dtj.transaction_source_tags
    , split_part(split_part(dtj.transaction_source_tags, 'RCID#', 2), '~', 1) AS external_account_number
    , split_part(split_part(dtj.transaction_source_tags, 'RINS#', 2), '~', 1) AS external_institution_id 
    , split_part(split_part(dtj.transaction_source_tags, 'ACHCOID#', 2), '~', 1) AS originating_company_id
    , split_part(split_part(dtj.transaction_source_tags, 'ACHID#', 2), '~', 1) AS external_institution_trans_id
    , split_part(split_part(dtj.transaction_source_tags, 'ACHODFI#', 2), '~', 1) AS originator_dfi_id
    , auth.merchant_name AS merchant_name
FROM tdm_bank.cleansed.profile_daily_transaction_journal dtj
    LEFT JOIN tdm_bank.cleansed.profile_authorizations auth ON auth.authorization_id = split_part(split_part(dtj.transaction_source_tags, 'AUTHID#', 2), '~', 1) AND dtj.transaction_source_tags LIKE '%%AUTHID#%%'
    JOIN tdm_bank.cleansed.profile_deposits dep ON dep.account_number = dtj.account_number
    JOIN tdm_bank.cleansed.profile_customers cif ON cif.customer_number = dep.customer_number
    LEFT JOIN tdm_bank.cleansed.profile_transaction_codes trn on dtj.external_transaction_code = trn.transaction_code
WHERE endbal IS NOT NULL
    and dtj.transaction_amount IS NOT NULL
    and dtj.transaction_amount NOT LIKE '%%#%%'
    
   and cif.sofi_user_id_reference = 406184
    and SYSTEM_PROCESSING_DATE =  '2020-01-24'
    
limit 5
""")

,SYSTEM_PROCESSING_DATE,CLOCK_TIME,BUSINESS_ACCOUNT_NUMBER,CREATED_DT,ENDBAL,TRANSACTION_CODE,TRANSACTION_AMOUNT,IS_CREDIT,USER_ID,TRANSACTION_SOURCE_TAGS,EXTERNAL_ACCOUNT_NUMBER,EXTERNAL_INSTITUTION_ID,ORIGINATING_COMPANY_ID,EXTERNAL_INSTITUTION_TRANS_ID,ORIGINATOR_DFI_ID,MERCHANT_NAME
0,2020-01-24,08:21:16,-4561875479856758071,2020-01-24,12026.57,ACHDWIN,25,0,406184,2062748232898124085,,,,,,None
1,2020-01-24,08:21:16,-4561875479856758071,2020-01-24,12026.57,ACHDWIN,25,0,406184,2062748232898124085,,,,,,None


postgres

In [8]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15556,database='sofi_profile_reporting',user='rarevalo@sofi.org',password=' ')

In [11]:
pd.read_sql(
"""-- banking transactions by business account.
SELECT tjd,
       dtj.time,
       dtj.cid AS business_account_number,
       cdt AS created_dt,
       endbal,
       dtj.etc AS transaction_code,
       tamt AS transaction_amount,
       trn.dc as is_credit,
       cif.zsofiid as user_id,
       split_part(split_part(tso, 'RCID#', 2), '~', 1) AS external_account_number,
       split_part(split_part(dtj.tso, 'RINS#', 2), '~', 1) AS external_institution_id,
       split_part(split_part(dtj.tso, 'ACHCOID#', 2), '~', 1) AS originating_company_id,
       split_part(split_part(dtj.tso, 'ACHID#', 2), '~', 1) AS external_institution_trans_id,
       split_part(split_part(dtj.tso, 'ACHODFI#', 2), '~', 1) AS originator_dfi_id,
       auth.merchnm AS merchant_name
FROM dtj
LEFT JOIN authdtl auth ON auth.authid = split_part(split_part(dtj.tso, 'AUTHID#', 2), '~', 1) AND dtj.tso LIKE '%%AUTHID#%%'
JOIN profile_reporting.dep ON dep.cid = dtj.cid
JOIN cif ON cif.acn = dep.acn
LEFT JOIN trn on dtj.etc = trn.etc
WHERE endbal IS NOT NULL
      and tamt IS NOT NULL
      and tamt NOT LIKE '%%#%%'     
      and cif.zsofiid = 406184
      and tjd = '2020-01-24'
limit 5     
""", conn)

,tjd,time,business_account_number,created_dt,endbal,transaction_code,transaction_amount,is_credit,user_id,external_account_number,external_institution_id,originating_company_id,external_institution_trans_id,originator_dfi_id,merchant_name
0,2020-01-24,06:28:06,410000147365,2020-01-24,12026.57,ACHDWIN,25,0,406184,,,9000088972,61433099,12204348,None


In [ ]:
%%time
transactions_postgres = pd.read_sql(
"""-- banking transactions by business account.
SELECT tjd,
       dtj.time,
       dtj.cid AS business_account_number,
       cdt AS created_dt,
       endbal,
       dtj.etc AS transaction_code,
       tamt AS transaction_amount,
       trn.dc as is_credit,
       cif.zsofiid as user_id,
       split_part(split_part(tso, 'RCID#', 2), '~', 1) AS external_account_number,
       split_part(split_part(dtj.tso, 'RINS#', 2), '~', 1) AS external_institution_id,
       split_part(split_part(dtj.tso, 'ACHCOID#', 2), '~', 1) AS originating_company_id,
       split_part(split_part(dtj.tso, 'ACHID#', 2), '~', 1) AS external_institution_trans_id,
       split_part(split_part(dtj.tso, 'ACHODFI#', 2), '~', 1) AS originator_dfi_id,
       auth.merchnm AS merchant_name
FROM dtj
LEFT JOIN authdtl auth ON auth.authid = split_part(split_part(dtj.tso, 'AUTHID#', 2), '~', 1) AND dtj.tso LIKE '%%AUTHID#%%'
JOIN profile_reporting.dep ON dep.cid = dtj.cid
JOIN cif ON cif.acn = dep.acn
LEFT JOIN trn on dtj.etc = trn.etc
WHERE endbal IS NOT NULL
      and tamt IS NOT NULL
      and tamt NOT LIKE '%%#%%'     
    
""", conn)

transactions_postgres.to_parquet('transactions_postgres.parquet.gzip', compression='gzip')

In [ ]:
describe_df(t)

### source_tags

In [14]:
 !pip install "snowflake-connector-python[pandas]" --quiet

import snowflake.connector

def get_snowflake_connection():
    ctx = snowflake.connector.connect(
        user="<username>",
        password="<password>",
        host="localhost",
        port=1444,
        account="sdm",
        warehouse="DATA_SCIENCE",
        database="DW_PRODUCTION",
        protocol="http"
    )
    return ctx


def vt(sql):
    with get_snowflake_connection() as ctx:
        with ctx.cursor() as cs:
            cs.execute(sql)
            allthedata = cs.fetchall()
            return allthedata
                


TSO columns are broken out into separate tables. The most frequently present tags are parsed into columns (appear in more than 20% of records) and there is a JSON column with all the record's tags converted to JSON. The join is on the key_hash column and the TSO columns live in tdm_bank.modeled.profile_{source table name}_transaction_source_tags

       split_part(split_part(tso, 'RCID#', 2), '~', 1) AS external_account_number,
       split_part(split_part(dtj.tso, 'RINS#', 2), '~', 1) AS external_institution_id,
       split_part(split_part(dtj.tso, 'ACHCOID#', 2), '~', 1) AS originating_company_id,
       split_part(split_part(dtj.tso, 'ACHID#', 2), '~', 1) AS external_institution_trans_id,
       split_part(split_part(dtj.tso, 'ACHODFI#', 2), '~', 1) AS originator_dfi_id,
       
        RCID# = recipient_account__rcid
        RINS#= recipient_institution__rins
        ACHCOID# = ach_company_id__achcoid
        ACHID# = ach_id__achid
        ACHODFI# = ach_originating_dfi_identification__achodfi

In [20]:
profile_tables = vt("""show tables like 'profile_%' in tdm_bank.modeled""")
[profile_tables[i][1] for i in range(len(profile_tables)) ]

['PROFILE_DAILY_MISCELLANEOUS_JOURNAL_TRANSACTION_SOURCE_TAGS',
 'PROFILE_DAILY_TRANSACTION_JOURNAL_TRANSACTION_SOURCE_TAGS',
 'PROFILE_EFT_OUTGOING_CLEARING_ITEMS_TRANSACTION_SOURCE_TAGS',
 'PROFILE_EFT_TRANSACTIONS_OUTBOUND_TRANSACTION_SOURCE_TAGS',
 'PROFILE_MOBILE_CHECK_DEPOSIT_TRANSACTION_SOURCE_TAGS',
 'PROFILE_PERMANENT_HOLDS_TRANSACTION_SOURCE_TAGS',
 'PROFILE_REJECTED_TRANSACTIONS_TRANSACTION_SOURCE_TAGS',
 'PROFILE_SOFI_TO_SOFI_PRODUCT_TRANSFERS_TRANSACTION_SOURCE_TAGS',
 'PROFILE_TELLER_TRANSACTIONS_TRANSACTION_SOURCE_TAGS',
 'PROFILE_TRANSACTION_AUTHORIZATIONS_TRANSACTION_SOURCE_TAGS',
 'PROFILE_TRANSACTION_HISTORY_TRANSACTION_SOURCE_TAGS',
 'PROFILE_VAULT_TRANSACTION_HISTORY_TRANSACTION_SOURCE_TAGS']

In [51]:
profile_tables = vt("""show tables like '%payroll%' in tdm_bank.cleansed""")
[profile_tables[i][1] for i in range(len(profile_tables)) ]

['BAAS_PAYROLL_ORG_DETAILS', 'PROFILE_INCOMING_PAYROLL_TRANSACTIONS_DETECTED']

    RCID# = recipient_account__rcid in table profile_eft_transactions
    RINS#= recipient_institution__rins in table profile_eft_transactions
    ACHCOID# = ach_company_id__achcoid in table profile_incoming_payroll_transactions_detected
    ACHID# = ach_id__achid - didn't find this one specifically but found ach_company_id in profile_incoming_payroll_transactions_detected
    ACHODFI# = ach_originating_dfi_identification__achodfi - didn't find this one

In [45]:
# RCID# = recipient_account__rcid
# run_query("""select 
# * from tdm_bank.modeled.PROFILE_EFT_TRANSACTIONS_OUTBOUND_TRANSACTION_SOURCE_TAGS limit 5""").T

run_query("""select 
KEY_HASH
,RECIPIENT_ACCOUNT__RCID
,recipient_institution__rins 
from tdm_bank.modeled.PROFILE_EFT_TRANSACTIONS_OUTBOUND_TRANSACTION_SOURCE_TAGS limit 5""").T

,0,1,2,3,4
KEY_HASH,5830745876843707022,5830755482905570651,5830756019222127442,5830757653758287249,5830760341099623941
RECIPIENT_ACCOUNT__RCID,********2594,248128152324,7167964563,851629139,5221704922
RECIPIENT_INSTITUTION__RINS,565296,103100195,121042882,103000648,031100649


In [33]:
#  ACHCOID# = ach_company_id__achcoid  ACH_COMPANY_ID

run_query("""select 
* from tdm_bank.cleansed.PROFILE_INCOMING_PAYROLL_TRANSACTIONS_DETECTED limit 5""").T

,0,1,2,3,4
ACCOUNT_NUMBER,-2295179619573967256,8125588691613919026,3769978474997888329,-6477066774788773020,7554947265565923009
ACH_COMPANY_ID,1741180155,5264681992,8581943813,1134994650,9111111101
ACTIVE_STATUS,0,0,0,0,0
AMOUNT_OF_FIRST_DEPOSIT,200,127.75,1943.47,142.31,100
CONFIRMED_IDENTIFIER,0,0,0,0,0
CREATED_AT,2022-07-01 06:42:15.830000,2022-07-01 06:42:15.830000,2022-07-01 06:42:15.830000,2022-07-01 06:42:15.830000,2022-07-01 06:42:15.830000
DATE_OF_FIRST_DEPOSIT,2019-01-24,2019-10-11,2020-09-15,2019-03-29,2019-01-18
INTERVAL_OF_DIRECT_DEPOSITS,bi-weekly,weekly,semi-monthly,bi-weekly,bi-weekly
MOST_RECENT_DATE,2019-03-07,2019-11-18,2020-10-30,2019-06-28,2019-03-01
ORIGINATOR_NAME,THE METHODIST HO,VENMO,AON SERVICE CORP,JPMORGAN CHASE B,SOCIAL FINANCE


In [53]:
run_query("""select * from tdm_bank.modeled.profile_daily_transaction_journal_transaction_source_tags limit 5""")
run_query("""select * from tdm_bank.modeled.profile_daily_transaction_journal_transaction_source_tags limit 5""")

,KEY_HASH,LOADED_AT,JSON_PAYLOAD,ACCOUNT_OWNERS_SOFIIDS__ZSOFIID,ACH_ABA_NUMBER__ACHABA,ACH_COMPANY__ACHCO,ACH_COMPANY_DESCRIPTIVE_DATE__ACHCDD,ACH_COMPANY_DISCRETIONARY_DATA__ACHDD,ACH_COMPANY_DISCRETIONARY_ID__ACHCODIS,ACH_COMPANY_ID__ACHCOID,...,EFT_PAYMENT_SEQUENCE__SEQ,ERROR_CORRECT_TRACE_NUMBER__EC,MERCHANT_TRADE_NAME__MTDNME,PAYMENT_ELEMENT__PE,STATE_PROVINCE__STPRV,SUPPORTING_INFORMATION_FILE__SIF,TRANSACTION_ID_FOR_DEEP_LINK__TRANSACTIONID,TRANSACTION_POSTED_BY_NAME_OF_OWNER__ZPOSTEDBYNAM,TRANSACTION_POSTED_BY_SOFI_ID_OF_OWNER__ZPOSTEDBY,TRANSACTION_TYPE__TRNTYPE
0,394170371672518846,2022-04-21 12:32:12.641,"{\n ""BATCH_OFFSET_TYPE__BOFFTYPE"": ""reg"",\n ...",None,None,None,None,None,None,None,...,None,None,None,reg,None,None,P-1000006451583-21,None,None,None
1,394170625845522229,2022-05-31 22:52:41.845,"{\n ""BATCH_OFFSET_TYPE__BOFFTYPE"": ""reg"",\n ...",None,None,None,None,None,None,None,...,None,None,None,reg,None,None,P-1000025358774-16,None,None,None
2,394170629122475354,2022-04-21 13:29:22.645,"{\n ""ACCOUNT_OWNERS_SOFIIDS__ZSOFIID"": ""18893...","18893719,18926357",None,None,None,None,None,None,...,None,None,BEDBATH&BEYOND# 200 Col,None,None,None,P-1000011558653-317,Jennifer Cimaglia-Brand,18893719,None
3,394170702761322569,2022-04-21 14:12:10.258,"{\n ""ACCOUNT_OWNERS_SOFIIDS__ZSOFIID"": ""20907...",20907883,None,None,None,None,None,None,...,None,None,SQ *GREENPOINT BEER &,POS,NY,None,P-1000014968744-358,Erik Erlandson,20907883,POS
4,394170834225395843,2022-04-21 13:11:08.634,"{\n ""ACCOUNT_OWNERS_SOFIIDS__ZSOFIID"": ""16569...",16569945,031101334,CAPITAL ONE,210729,S,None,9279744980,...,None,WEB,None,None,None,10175^2997^18^65955,P-1000004209296-3598,Juliannie Richardson,16569945,None


In [5]:
run_query(
"""
with outputs as (
    select *
    from tdm_bank.cleansed.profile_daily_transaction_journal j
    left join tdm_bank.modeled.profile_daily_transaction_journal_transaction_source_tags jst
        on j.key_hash = jst.key_hash
    )
select
ach_company_id__achcoid AS originating_company_id,
ach_id__achid AS external_institution_trans_id,
ach_originating_dfi_identification__achodfi AS originator_dfi_id,
o.json_payload:RECIPIENT_ACCOUNT__RCID::varchar as external_account_number,
o.json_payload:RECIPIENT_INSTITUTION__RINS::varchar as external_institution_id
from outputs o limit 5""" )

,ORIGINATING_COMPANY_ID,EXTERNAL_INSTITUTION_TRANS_ID,ORIGINATOR_DFI_ID,EXTERNAL_ACCOUNT_NUMBER,EXTERNAL_INSTITUTION_ID
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,None,None,None,None,None


## Threat_metrix

Not used in the model

In [276]:
run_query("""
-- Query for Threat Metrix data pull
with
    map
    as
    (
        select distinct thirdparty_raw_id, event_id
        from TDM_RISK_MGMT_HUB.CLEANSED.decision
    )
select e.user_id,
       tpr.created_dt,
       parse_json(tpr.response):results.os as os,
       parse_json(tpr.response):results.time_zone as time_zone,
       parse_json(tpr.response):results.dns_ip_geo as dns_ip_geo,
       parse_json(tpr.response):results.enabled_ck as enabled_ck,
       parse_json(tpr.response):results.enabled_fl as enabled_fl,
       parse_json(tpr.response):results.enabled_im as enabled_im,
       parse_json(tpr.response):results.enabled_js as enabled_js,
       parse_json(tpr.response):results.screen_res as screen_res,
       parse_json(tpr.response):results.agent_brand as agent_brand,
       parse_json(tpr.response):results.device_name  as device_name,
       parse_json(tpr.response):results.page_time_on as page_time_on,
       parse_json(tpr.response):results.dns_ip_region as dns_ip_region,
       parse_json(tpr.response):results.agent_language as agent_language,
       parse_json(tpr.response):results.tmx_risk_rating as tmx_risk_rating,
       parse_json(tpr.response):results.browser_language as browser_language
from TDM_RISK.CLEANSED.event e
    join map on e.event_id=map.event_id
    join TDM_RISK.CLEANSED.thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
    join TDM_RISK.CLEANSED.thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='TMX'



limit 5"""
         )

,USER_ID,CREATED_DT,OS,TIME_ZONE,DNS_IP_GEO,ENABLED_CK,ENABLED_FL,ENABLED_IM,ENABLED_JS,SCREEN_RES,AGENT_BRAND,DEVICE_NAME,PAGE_TIME_ON,DNS_IP_REGION,AGENT_LANGUAGE,TMX_RISK_RATING,BROWSER_LANGUAGE
0,22666868,2022-03-05 10:04:01.309000-08:00,"[\n ""Android""\n]","[\n ""-360""\n]","[\n ""US""\n]","[\n ""yes""\n]","[\n ""no""\n]","[\n ""no""\n]","[\n ""no""\n]","[\n ""2400x1080""\n]","[\n ""samsung, samsung""\n]","[\n ""Galaxy S20 FE 5G""\n]",None,"[\n ""wisconsin""\n]","[\n ""en-us""\n]","[\n ""neutral""\n]","[\n ""en-US""\n]"
1,14399099,2019-08-29 17:19:25.088000-07:00,"[\n ""Android""\n]","[\n ""-300""\n]","[\n ""US""\n]","[\n ""yes""\n]","[\n ""no""\n]","[\n ""yes""\n]","[\n ""yes""\n]","[\n ""2880x1442""\n]",None,None,"[\n ""2247""\n]","[\n ""new jersey""\n]",None,"[\n ""neutral""\n]","[\n ""en-US,en;q=0.9""\n]"
2,14947506,2019-11-04 17:57:16.534000-08:00,"[\n ""iOS""\n]","[\n ""-360""\n]","[\n ""US""\n]","[\n ""yes""\n]","[\n ""no""\n]","[\n ""yes""\n]","[\n ""yes""\n]","[\n ""2436x1125""\n]",None,None,"[\n ""43456""\n]","[\n ""minnesota""\n]",None,"[\n ""neutral""\n]","[\n ""en-us""\n]"
3,16464100,2020-08-04 17:48:45.037000-07:00,"[\n ""Android""\n]","[\n ""-300""\n]","[\n ""US""\n]","[\n ""yes""\n]","[\n ""no""\n]","[\n ""no""\n]","[\n ""no""\n]","[\n ""2280x1080""\n]","[\n ""samsung, samsung""\n]","[\n ""JLM Interior Auto Detailing""\n]",None,"[\n ""new york""\n]","[\n ""en-us""\n]","[\n ""neutral""\n]","[\n ""en-US""\n]"
4,17626987,2021-11-20 12:22:30.529000-08:00,"[\n ""Android""\n]","[\n ""-420""\n]","[\n ""US""\n]","[\n ""yes""\n]","[\n ""no""\n]","[\n ""no""\n]","[\n ""no""\n]","[\n ""2340x1080""\n]","[\n ""samsung, samsung""\n]","[\n ""Galaxy A50""\n]",None,"[\n ""california""\n]","[\n ""en-us""\n]","[\n ""neutral""\n]","[\n ""en-US""\n]"


## Giact

In [283]:
run_query(
    
"""-- Query for Giact data pulls
with
    map
    as
    (
        select distinct thirdparty_raw_id, event_id
        from TDM_RISK_MGMT_HUB.CLEANSED.decision
    )
select e.user_id, 
       tpr.created_dt,
       parse_json(tpr.request):sofiAccountNumber as business_account_number,
       parse_json(tpr.response):bankName as giact_bank_name,
       parse_json(tpr.response):createdDate as giact_created_date,
       parse_json(tpr.response):itemReferenceId as giact_item_reference_id,
       parse_json(tpr.response):accountAddedDate as giact_account_added_date,
       parse_json(tpr.response):accountResponseCode as giact_account_response_code,
       parse_json(tpr.response):customerResponseCode as giact_customer_response_code, 
       parse_json(tpr.response):verificationResponse as giact_verification_response,
       parse_json(tpr.response):accountLastUpdatedDate as giact_account_last_updated_date
from TDM_RISK.CLEANSED.event e
    join map on e.event_id=map.event_id
    join TDM_RISK.CLEANSED.thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
    join TDM_RISK.CLEANSED.thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='GIACT'

limit 5 

""")




,USER_ID,CREATED_DT,BUSINESS_ACCOUNT_NUMBER,GIACT_BANK_NAME,GIACT_CREATED_DATE,GIACT_ITEM_REFERENCE_ID,GIACT_ACCOUNT_ADDED_DATE,GIACT_ACCOUNT_RESPONSE_CODE,GIACT_CUSTOMER_RESPONSE_CODE,GIACT_VERIFICATION_RESPONSE,GIACT_ACCOUNT_LAST_UPDATED_DATE
0,10397945,2019-06-17 07:46:39.014000-07:00,"""410001839512""","""REGIONS BANK""",1560782798367,10135535882,1263877200000,"""_1111""","""CA_11""","""PASS""",1560312000000
1,17975421,2021-01-30 07:17:30.131000-08:00,"""410012467911""","""JPMORGAN CHASE""",1612019849658,10318524324,1037682000000,"""_1111""","""CA_11""","""PASS""",1037682000000
2,16202566,2020-07-09 04:55:13.260000-07:00,"""410005421748""","""WELLS FARGO BANK""",1594295711443,10245063931,1586145600000,"""RT01""","""NULL""","""DECLINED""",1586232000000
3,14255362,2019-07-26 16:03:08.713000-07:00,"""410002213451""","""BANK OF AMERICA N.A.""",1564182188962,10143177738,1339992000000,"""_5555""","""CA_11""","""PASS""",1355806800000
4,16352746,2020-08-29 20:42:36.803000-07:00,"""410005913076""","""JPMORGAN CHASE""",1598758956398,10264490556,1501732800000,"""_1111""","""CA_21""","""RISK_ALERT""",1517288400000


postgres

In [39]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15722,database='sofi_fraud',user='rarevalo@sofi.org',password=' ')

In [286]:
pd.read_sql("""-- Query for Giact data pulls
with map as (select distinct thirdparty_raw_id, event_id
             from decision)
select e.user_id, 
       tpr.created_dt,
       tpr.request::jsonb->>'sofiAccountNumber' as business_account_number,
       tpr.response::jsonb->>'bankName' as giact_bank_name,
       tpr.response::jsonb->>'createdDate' as giact_created_date,
       tpr.response::jsonb->>'itemReferenceId' as giact_item_reference_id,
       tpr.response::jsonb->>'accountAddedDate' as giact_account_added_date,
       tpr.response::jsonb->>'accountResponseCode' as giact_account_response_code,
       tpr.response::jsonb->>'customerResponseCode' as giact_customer_response_code, 
       tpr.response::jsonb->>'verificationResponse' as giact_verification_response,
       tpr.response::jsonb->>'accountLastUpdatedDate' as giact_account_last_updated_date
from event e
join map on e.event_id=map.event_id
join thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
join thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='GIACT'

limit 5 
""",conn)

,user_id,created_dt,business_account_number,giact_bank_name,giact_created_date,giact_item_reference_id,giact_account_added_date,giact_account_response_code,giact_customer_response_code,giact_verification_response,giact_account_last_updated_date
0,14459552,2019-10-01 18:32:37.176588+00:00,410002840238,NAVY ARMY COMMUNITY CREDIT UNION,1569954756969,10156654357,None,_3333,NULL,PASS,None
1,14881218,2019-10-26 20:41:34.019819+00:00,410003143946,TCF NATIONAL BANK ILLINOIS,1572122494007,10161667130,1119240000000,_1111,CA_11,PASS,1552622400000
2,13528145,2019-04-03 21:37:22.673049+00:00,410001416170,JPMORGAN CHASE BANK NA,1554327441870,10121590137,None,RT00,NULL,DECLINED,None
3,13939477,2019-06-12 02:39:46.166286+00:00,410001812088,BANK OF AMERICA N.A.,1560307185403,10134666677,1526529600000,_1111,CA_11,PASS,1542085200000
4,18119449,2021-02-09 00:59:10.474811+00:00,410012818196,THE BANCORP BANK,1612832350165,10323121650,None,_3333,NULL,PASS,None


###### CHECK

In [40]:
%%time
giact_postgres = pd.read_sql("""-- Query for Giact data pulls
with map as (select distinct thirdparty_raw_id, event_id
             from decision)
select e.user_id, 
       tpr.created_dt,
       tpr.request::jsonb->>'sofiAccountNumber' as business_account_number,
       tpr.response::jsonb->>'bankName' as giact_bank_name,
       tpr.response::jsonb->>'createdDate' as giact_created_date,
       tpr.response::jsonb->>'itemReferenceId' as giact_item_reference_id,
       tpr.response::jsonb->>'accountAddedDate' as giact_account_added_date,
       tpr.response::jsonb->>'accountResponseCode' as giact_account_response_code,
       tpr.response::jsonb->>'customerResponseCode' as giact_customer_response_code, 
       tpr.response::jsonb->>'verificationResponse' as giact_verification_response,
       tpr.response::jsonb->>'accountLastUpdatedDate' as giact_account_last_updated_date
from event e
join map on e.event_id=map.event_id
join thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
join thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='GIACT'

""",conn)

CPU times: user 3.36 s, sys: 225 ms, total: 3.58 s
Wall time: 12min 45s


In [138]:
giact_postgres.describe(include='all')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  if __name__ == '__main__':


,user_id,created_dt,business_account_number,giact_bank_name,giact_created_date,giact_item_reference_id,giact_account_added_date,giact_account_response_code,giact_customer_response_code,giact_verification_response,giact_account_last_updated_date
count,505579,505579,383020,504478,505153,505153,248004,504798,505153,505136,249569
unique,246990,505577,182217,6395,505012,505150,5222,12,7,8,5280
top,13155542,2020-03-13 12:18:57.674735+00:00,410000549070,THE BANCORP BANK,1595813274156,0,1526875200000,_1111,NULL,PASS,1537502400000
freq,324,2,296,35550,5,4,3814,206786,256669,271334,2074
first,NaN,2019-01-16 16:06:04.834669+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2022-07-01 15:09:03.439845+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
describe_df(giact_postgres)

----------------------------------------
Columns: 11
Rows: 505877
Memory usage: (301.42 Mb)
----------------------------------------
NUMERICAL VARIABLES:
 MIN   | MEAN   | MAX   | MODE   | %NULLS   | #_NULLS   | NOT_NULL
-------+--------+-------+--------+----------+-----------+------------
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                                 |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+---------------------------------+----------+-----------+------------+----------------------
 1  | giact_account_added_date        | 50.95%   | 257735    |     248142 | 5224 unique values
 2  | giact_account_last_updated_date | 50.64%   | 256170    |     249707 | 5282 unique values
 3  | business_account_number         | 24.29%   | 122857    |     383020 | 182217 unique values
 4  | giact_bank_name                 | 0.22%    | 1101      |     504776 | 6397 unique values
 5  | giact_ac

In [42]:
%%time
giact_sf = run_query(
    
"""-- Query for Giact data pulls
with
    map
    as
    (
        select distinct thirdparty_raw_id, event_id
        from TDM_RISK_MGMT_HUB.CLEANSED.decision
    )
select e.user_id, 
       tpr.created_dt,
       parse_json(tpr.request):sofiAccountNumber as business_account_number,
       parse_json(tpr.response):bankName as giact_bank_name,
       parse_json(tpr.response):createdDate as giact_created_date,
       parse_json(tpr.response):itemReferenceId as giact_item_reference_id,
       parse_json(tpr.response):accountAddedDate as giact_account_added_date,
       parse_json(tpr.response):accountResponseCode as giact_account_response_code,
       parse_json(tpr.response):customerResponseCode as giact_customer_response_code, 
       parse_json(tpr.response):verificationResponse as giact_verification_response,
       parse_json(tpr.response):accountLastUpdatedDate as giact_account_last_updated_date
from TDM_RISK.CLEANSED.event e
    join map on e.event_id=map.event_id
    join TDM_RISK.CLEANSED.thirdparty_raw tpr on tpr.thirdparty_raw_id = map.thirdparty_raw_id
    join TDM_RISK.CLEANSED.thirdparty_provider tpp on tpr.thirdparty_provider_id=tpp.thirdparty_provider_id
where e.kob='BANKING' and tpp.code='GIACT'


""")




CPU times: user 2.9 s, sys: 477 ms, total: 3.37 s
Wall time: 50.2 s


In [43]:
describe_df(giact_sf)

----------------------------------------
Columns: 11
Rows: 505876
Memory usage: (309.87 Mb)
----------------------------------------
NUMERICAL VARIABLES:
 MIN   | MEAN   | MAX   | MODE   | %NULLS   | #_NULLS   | NOT_NULL
-------+--------+-------+--------+----------+-----------+------------
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                                 |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+---------------------------------+----------+-----------+------------+----------------------
 1  | GIACT_ACCOUNT_ADDED_DATE        | 50.95%   | 257735    |     248141 | 5224 unique values
 2  | GIACT_ACCOUNT_LAST_UPDATED_DATE | 50.64%   | 256170    |     249706 | 5282 unique values
 3  | BUSINESS_ACCOUNT_NUMBER         | 24.29%   | 122856    |     383020 | 182217 unique values
 4  | GIACT_BANK_NAME                 | 0.22%    | 1101      |     504775 | 6397 unique values
 5  | GIACT_AC

In [137]:
from IPython.display import Audio
wave = np.sin(8*np.pi*500*np.arange(10000*0.15)/10000)
Audio(wave, rate=10000, autoplay=True)

### user_metadata_dw_1 

In [14]:
run_query("""
WITH
    bid
    AS
    (
        SELECT DISTINCT badf.borrower_id
        FROM sofi_dw.sofidw.banking_account_daily_facts badf
            JOIN sofi_dw.sofidw.banking_accounts ba ON badf.banking_account_id = ba.banking_account_id
        WHERE product_type='SoFi Money Cash Account'
            AND ba.active_flag=TRUE
    )
SELECT bid.borrower_id,
    b.user_id,
    b.date_of_birth,
    b.sofi_employee_ind,
    b.first_name,
    b.last_name,
    b.current_addr_line_1
FROM bid
    JOIN sofi_dw.sofidw.borrowers b ON b.borrower_id = bid.borrower_id
 limit 5""")

,BORROWER_ID,USER_ID,DATE_OF_BIRTH,SOFI_EMPLOYEE_IND,FIRST_NAME,LAST_NAME,CURRENT_ADDR_LINE_1
0,7154453,13343100,1989-02-22,false,Kristin,Myers,11186 Jerryson Drive
1,16648562,22839445,1972-09-06,false,NADA,ABI-KARAM,9197 Ramblewood Drive
2,10489204,16667095,1979-05-10,false,Dhammika,VIYALAMUNNA,9600 Royal Ln
3,10127260,16306101,1973-07-28,false,David,Hamrick,127 Meadow Lane
4,9852916,16033074,1991-02-15,false,Daniel,Stewart,415 Berkeley Hill Drive


CHECK

In [16]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15501,database='sofi_dw',user='rarevalo@sofi.org',password=' ')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [19]:
%%time 
user_metadata_dw_1_postgres =  pd.read_sql("""-- user_id/borrower_id level metadata (not for use in model)
WITH bid AS
    (SELECT DISTINCT badf.borrower_id
     FROM banking_account_daily_facts badf
     JOIN banking_accounts ba ON badf.banking_account_id = ba.banking_account_id
     WHERE product_type='SoFi Money Cash Account'
     AND ba.active_flag=TRUE)
SELECT bid.borrower_id,
       b.user_id,
       b.date_of_birth,
       b.sofi_employee_ind,
       b.first_name,
       b.last_name,
       b.current_addr_line_1
FROM bid
JOIN borrowers b ON b.borrower_id = bid.borrower_id""", conn)

CPU times: user 2.93 s, sys: 539 ms, total: 3.47 s
Wall time: 25min 1s


In [29]:
from tabulate import tabulate
import sys
def describe_df(df,floatfmt= '.3f'):
    # Numerical
    print("--" * 20)
    print('Columns:', df.shape[1])
    print('Rows:', df.shape[0])
    print("Memory usage:", (f"({(sys.getsizeof(df) / 1024 ** 2):.2f} Mb)"))

    print("--" * 20)
    print('NUMERICAL VARIABLES:')

    numerical = df.select_dtypes(include=np.number)
    concatenated_numerical = pd.concat([
        round(numerical.isnull().sum() / df.shape[0] * 100, 2).astype(str) + "%",
        numerical.isnull().sum(),
        numerical.count(),
        numerical.min(),
        numerical.mean(),
        numerical.max()
    ], axis=1, keys=["%NULLS", "COUNT_NULLS", "NOT_NULL", 'MIN', 'MEAN', 'MAX'], sort=False).sort_values('COUNT_NULLS',
                                                                                                         ascending=False).reset_index().rename(
        columns={'index': ''})

    t = numerical.mode().T
    t.rename(columns={0: 'MODE'}, inplace=True)
    concatenated_numerical = concatenated_numerical.merge(t, how='left', left_on='', right_on=t.index)
    concatenated_numerical.index = concatenated_numerical.index + 1
    concatenated_numerical = concatenated_numerical.iloc[:, [0, 4, 5, 6, 7, 1, 2, 3]]

    print(tabulate(concatenated_numerical, headers=
    [
        'MIN',
        'MEAN',
        'MAX',
        'MODE',
        "%NULLS",
        "#_NULLS",
        "NOT_NULL",

    ], tablefmt="presto", colalign=("right"), floatfmt=floatfmt))

    # Categorical

    print('-----' * 20)
    print()
    print('CATEGORICAL VARIABLES:')
    categorical = df.select_dtypes(['object','category'])
    if categorical.shape[1] == 0:
        print("No Categorical Variables")
    else:
        concatenated_categorical = pd.concat([

            round(categorical.isnull().sum() / df.shape[0] * 100, 2).astype(str) + "%",
            categorical.isnull().sum(),
            categorical.count()
        ],

            keys=["%NULLS",
                  "COUNT_NULLS",
                  "NOT_NULL"], axis=1, sort=False).sort_values('%NULLS', ascending=False).reset_index().rename(
            columns={'index': ''})

        max_unique = 5
        u_strs = []

        for col in categorical:
            series = categorical.loc[categorical[col].notnull(), col]
            n_unique = series.nunique()
            if n_unique > max_unique:
                u_strs.append(str(n_unique) + ' unique values')
            else:
                u_strs.append(str(series.unique()))

        t = pd.DataFrame(u_strs, categorical.columns)
        t = t.reset_index()
        t = t.rename(columns={'index': '', 0: 'Unique_Values'})
        concatenated_categorical = concatenated_categorical.merge(t, on='')
        concatenated_categorical.index = concatenated_categorical.index + 1

        print(tabulate(concatenated_categorical, headers=
        [
            "%NULLS",
            "#_NULLS",
            "NOT_NULL",
            "Unique_Values"

        ], tablefmt="presto", colalign=("left")))

In [30]:
describe_df(user_metadata_dw_1_postgres)

----------------------------------------
Columns: 7
Rows: 1593866
Memory usage: (393.61 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |             |   MIN |         MEAN |      MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+-------------+-------+--------------+----------+--------+----------+-----------+------------
 1  | borrower_id | 44    | 11816639.202 | 18479407 |     44 | 0.0%     |         0 |    1593866
 2  | user_id     | 0     | 17837654.824 | 24656702 |      0 | 0.0%     |         0 |    1593866
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                     |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+---------------------+----------+-----------+------------+-----------------------
 1  | date_of_birth       | 0.0%     | 4         |    1593862 | 24352 unique values
 2  | first_name          | 0.0%     | 0         |    1593866 | 198968 unique value

In [20]:
%%time
user_metadata_dw_1_sf = run_query("""
WITH
    bid
    AS
    (
        SELECT DISTINCT badf.borrower_id
        FROM sofi_dw.sofidw.banking_account_daily_facts badf
            JOIN sofi_dw.sofidw.banking_accounts ba ON badf.banking_account_id = ba.banking_account_id
        WHERE product_type='SoFi Money Cash Account'
            AND ba.active_flag=TRUE
    )
SELECT bid.borrower_id,
    b.user_id,
    b.date_of_birth,
    b.sofi_employee_ind,
    b.first_name,
    b.last_name,
    b.current_addr_line_1
FROM bid
    JOIN sofi_dw.sofidw.borrowers b ON b.borrower_id = bid.borrower_id
 """)

CPU times: user 3.49 s, sys: 752 ms, total: 4.24 s
Wall time: 16.6 s


In [31]:
describe_df(user_metadata_dw_1_sf)

----------------------------------------
Columns: 7
Rows: 1593866
Memory usage: (486.33 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |             |   MIN |         MEAN |      MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+-------------+-------+--------------+----------+--------+----------+-----------+------------
 1  | BORROWER_ID | 44    | 11816639.202 | 18479407 |     44 | 0.0%     |         0 |    1593866
 2  | USER_ID     | 0     | 17837654.824 | 24656702 |      0 | 0.0%     |         0 |    1593866
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                     |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+---------------------+----------+-----------+------------+-----------------------
 1  | DATE_OF_BIRTH       | 0.0%     | 4         |    1593862 | 24352 unique values
 2  | SOFI_EMPLOYEE_IND   | 0.0%     | 0         |    1593866 | ['false' 'true']
 3

In [34]:
%%time 
user_metadata_dw_1_sf_2 = run_query("""
WITH
    bid
    AS
    (
        SELECT DISTINCT badf.borrower_id
        FROM sofi_dw.sofidw.banking_account_daily_facts badf
            JOIN sofi_dw.sofidw.banking_accounts ba ON badf.banking_account_id = ba.banking_account_id
        WHERE product_type='SoFi Money Cash Account'
            AND ba.active_flag=TRUE
    )
SELECT bid.borrower_id,
    b.user_id,
    b.date_of_birth,
    b.sofi_employee_ind,
    b.first_name,
    b.last_name,
    b.current_addr_line_1
FROM bid
    JOIN TDM_RISK_MGMT_HUB.modeled.borrowers b ON b.borrower_id = bid.borrower_id
 """)

CPU times: user 3.32 s, sys: 723 ms, total: 4.05 s
Wall time: 16.8 s


In [35]:
describe_df(user_metadata_dw_1_sf_2)

----------------------------------------
Columns: 7
Rows: 1593866
Memory usage: (486.33 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |             |   MIN |         MEAN |      MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+-------------+-------+--------------+----------+--------+----------+-----------+------------
 1  | BORROWER_ID | 44    | 11816639.202 | 18479407 |     44 | 0.0%     |         0 |    1593866
 2  | USER_ID     | 0     | 17837654.824 | 24656702 |      0 | 0.0%     |         0 |    1593866
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                     |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+---------------------+----------+-----------+------------+-----------------------
 1  | DATE_OF_BIRTH       | 0.0%     | 4         |    1593862 | 24352 unique values
 2  | SOFI_EMPLOYEE_IND   | 0.0%     | 0         |    1593866 | ['false' 'true']
 3

#### banking_account_restrictions

In [9]:
%%time 

banking_account_restrictions_sf = run_query("""WITH
    last_unrestricted_date
    AS
    (
        SELECT business_account_number,
            max(effective_date) AS last_unrestricted_date
        FROM sofi_dw.sofidw.banking_accounts
        WHERE account_restricted_ind=FALSE
            AND active_flag=TRUE
        GROUP BY 1
    ),
    first_restricted_by_risk_date
    AS
    (
        SELECT business_account_number,
            min(effective_date) AS first_restricted_by_risk_date
        FROM    sofi_dw.sofidw.banking_accounts
        WHERE account_restricted_reason LIKE '%No%'
            AND active_flag=TRUE
        GROUP BY 1
    )
SELECT ba.business_account_number,
    max(last_unrestricted_date) AS last_unrestricted_date,
    min(first_restricted_by_risk_date) AS first_restricted_by_risk_date
FROM sofi_dw.sofidw.banking_accounts ba
    LEFT JOIN last_unrestricted_date lud ON lud.business_account_number=ba.business_account_number
    LEFT JOIN first_restricted_by_risk_date frbrd ON frbrd.business_account_number=ba.business_account_number
GROUP BY 1
""")

CPU times: user 1.94 s, sys: 484 ms, total: 2.42 s
Wall time: 8.08 s


In [10]:
describe_df(banking_account_restrictions_sf)

----------------------------------------
Columns: 3
Rows: 3688890
Memory usage: (278.75 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                         |   MIN |             MEAN |          MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+-------------------------+-------+------------------+--------------+--------+----------+-----------+------------
 1  | BUSINESS_ACCOUNT_NUMBER | 0     | 414404433426.533 | 960001270640 |      0 | 0.0%     |         0 |    3688890
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                               |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+-------------------------------+----------+-----------+------------+-------------------
 1  | LAST_UNRESTRICTED_DATE        | 4.78%    | 176187    |    3512703 | 323 unique values
 2  | FIRST_RESTRICTED_BY_RISK_DATE | 100.0%   | 3688834   |         56 | 29 unique values


In [11]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15501,database='sofi_dw',user='rarevalo@sofi.org',password=' ')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [12]:
banking_account_restrictions_postgres = pd.read_sql(
"""WITH last_unrestricted_date AS
    ( SELECT business_account_number,
             max(effective_date) AS last_unrestricted_date
     FROM banking_accounts
     WHERE account_restricted_ind=FALSE
         AND active_flag=TRUE
     GROUP BY 1 ),
     first_restricted_by_risk_date AS
    ( SELECT business_account_number,
             min(effective_date) AS first_restricted_by_risk_date
     FROM banking_accounts
     WHERE account_restricted_reason LIKE '%No%'
         AND active_flag=TRUE
     GROUP BY 1 )
SELECT ba.business_account_number,
       max(last_unrestricted_date) AS last_unrestricted_date,
       min(first_restricted_by_risk_date) AS first_restricted_by_risk_date
FROM banking_accounts ba
LEFT JOIN last_unrestricted_date lud ON lud.business_account_number=ba.business_account_number
LEFT JOIN first_restricted_by_risk_date frbrd ON frbrd.business_account_number=ba.business_account_number
GROUP BY 1""",conn)

In [13]:
describe_df(banking_account_restrictions_postgres)

----------------------------------------
Columns: 3
Rows: 3688890
Memory usage: (250.61 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                         |   MIN |             MEAN |          MAX |   MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+-------------------------+-------+------------------+--------------+--------+----------+-----------+------------
 1  | business_account_number | 0     | 414404433426.488 | 960001270640 |      0 | 0.0%     |         0 |    3688890
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                               |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+-------------------------------+----------+-----------+------------+-------------------
 1  | last_unrestricted_date        | 4.78%    | 176187    |    3512703 | 323 unique values
 2  | first_restricted_by_risk_date | 100.0%   | 3688834   |         56 | 29 unique values


### TDM_RISK_MGMT_HUB.modeled.banking_account_daily_facts 

In [2]:
import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15501,database='sofi_dw',user='rarevalo@sofi.org',password=' ')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 15501?


In [16]:
run_query("""select * from TDM_RISK_MGMT_HUB.modeled.banking_account_daily_facts limit 5 """).T

# sofi_dw.sofidw.banking_account_daily_facts
# sofi_dw.sofidw.banking_accounts

,0,1,2,3,4
ACCOUNT_AGE,486,710,151,484,766
ADDRESS_ID,16655330,10752675,18219147,17058951,17058951
AVERAGE_CURRENT_MONTH_ACCOUNT_BALANCE,1.08,16.96,0,0,18066.1
BANKING_ACCOUNT_ID,230987782,169904387,249226630,170175917,169886826
BANKING_CARD_ID_1,55146937,54494378,0,0,53766406
BORROWER_ID,1951636,1951569,1951544,1951194,1951194
CREATED_BY,sofidw,sofidw,sofidw,sofidw,sofidw
CREATED_DATE,2021-01-17 05:29:37.581648,2021-01-17 05:29:37.581648,2021-01-17 05:29:37.581648,2021-01-17 05:29:37.581648,2021-01-17 05:29:37.581648
CREDIT_PULL_ID,11672243,0,14763843,0,0
CURRENT_ACCOUNT_BALANCE,1.08,16.96,0,0,17472.2


In [14]:
run_query("""select * from TDM_RISK_MGMT_HUB.modeled.banking_accounts limit 5 """).T

,0,1,2,3,4
ACCOUNT_CLOSED_DATE,2019-04-01,None,None,None,2019-10-03
ACCOUNT_CLOSED_REASON,None,None,None,None,None
ACCOUNT_INTEREST_RATE,0,0,0,0,0
ACCOUNT_NICKNAME,,,joint,Schwab Personal,TD Bank
ACCOUNT_OPEN_DATE,2019-03-28,2019-04-19,2019-04-19,2019-04-19,2019-04-19
ACCOUNT_OWNER,Sofi Money,Sofi Money,Sofi Money,Sofi Money,Sofi Money
ACCOUNT_RESTRICTED_IND,false,false,false,false,false
ACCOUNT_RESTRICTED_REASON,None,None,None,None,None
ACCOUNT_STATUS,Closed,Active,Active,Active,Closed
ACCOUNT_TYPE_DESC,Single,Single,Single,Single,Single


In [5]:
run_query("""select * from TDM_RISK_MGMT_HUB.CLEANSED.thirdparty_raw
 limit 5""") 

,CREATED_BY,CREATED_DT,DMS_EXPORT_CHANGE_SEQ,DMS_EXPORT_TIMESTAMP,OP,REQUEST,RESPONSE,S3_FILENAME,SNOWFLAKE_RAW_TIMESTAMP,STATUS,THIRDPARTY_PROVIDER_ID,THIRDPARTY_RAW_ID,UPDATED_BY,UPDATED_DT
0,hshelby613@gmail.com,2022-06-20 10:38:25.330000-07:00,,2022-06-22 10:47:48.585521-07:00,None,"{""policy"": ""default"", ""modules"": [], ""event_ty...","{""results"": {""os"": [""Android""], ""ua_os"": [""and...",dms/fraud/fraud/thirdparty_raw/LOAD00000063.pa...,2022-06-22 11:26:48.509000-07:00,200,94f1be01-a093-442e-bd15-fab6f3e19f0e,8323c5ae-01d0-4f9a-b90d-05f474c1cd7b,None,NaT
1,delialeeanne0@gmail.com,2022-06-20 10:13:44.017000-07:00,,2022-06-22 10:47:48.585521-07:00,None,"{""policy"": ""default"", ""modules"": [], ""event_ty...","{""results"": {""org_id"": [""oiwd0wpz""], ""policy"":...",dms/fraud/fraud/thirdparty_raw/LOAD00000063.pa...,2022-06-22 11:26:48.509000-07:00,200,94f1be01-a093-442e-bd15-fab6f3e19f0e,b240a6cc-9c5f-43e5-852b-99fc0c881304,None,NaT
2,fraud@sofi.org,2022-06-21 11:09:13.772000-07:00,,2022-06-22 10:47:48.585521-07:00,None,"{""city"": ""Riverdale"", ""state"": ""NJ"", ""zipCode""...","{""id"": ""us_ver_97182743ccd54dd49324"", ""object""...",dms/fraud/fraud/thirdparty_raw/LOAD00000066.pa...,2022-06-22 11:28:06.798000-07:00,200,a815f2a8-a832-4cc7-a9fa-112a8696cb8c,b4e49800-bdf3-4411-a82f-ce3f6d89f046,None,NaT
3,lkfjunkmail@gmail.com,2022-06-21 12:27:16.055000-07:00,,2022-06-22 10:47:48.585521-07:00,None,"{""person"": {""SSN"": {""type"": ""ssn9"", ""number"": ...","{""status"": {""reference"": ""24492053"", ""requestI...",dms/fraud/fraud/thirdparty_raw/LOAD00000067.pa...,2022-06-22 11:28:06.798000-07:00,200,6dc16993-ea94-45d8-a0e4-19b95bf1988e,d000fefe-4cd0-4696-b40f-25a6a6fe7e76,None,NaT
4,Jerrylee6619@gmail.com,2022-06-21 13:18:38.819000-07:00,,2022-06-22 10:47:48.585521-07:00,None,"{""policy"": ""default"", ""modules"": [], ""event_ty...","{""results"": {""os"": [""Android""], ""js_os"": [""And...",dms/fraud/fraud/thirdparty_raw/LOAD00000067.pa...,2022-06-22 11:28:06.798000-07:00,200,94f1be01-a093-442e-bd15-fab6f3e19f0e,c1dcf155-9467-4338-8f23-cbeb6279ef9e,None,NaT


In [6]:
run_query("""select * from TDM_RISK.CLEANSED.thirdparty_raw
 limit 5""") 

,THIRDPARTY_RAW_ID,CREATED_DT,UPDATED_DT,THIRDPARTY_PROVIDER_ID,RESPONSE,REQUEST,STATUS,CREATED_BY,UPDATED_BY,DMS_EXPORT_TIMESTAMP,DMS_EXPORT_CHANGE_SEQ,OP,SNOWFLAKE_RAW_TIMESTAMP,S3_FILENAME
0,5efc0793-dae0-4386-a7e2-cba49e423eee,2022-05-26 23:47:28.427000-07:00,NaT,94f1be01-a093-442e-bd15-fab6f3e19f0e,"{""results"": {""os"": [""Android""], ""js_os"": [""And...","{""policy"": ""default"", ""modules"": [], ""event_ty...",200,tamand1964@gmail.com,None,2022-06-22 10:47:48.585521-07:00,,None,2022-06-22 11:01:00.606000-07:00,dms/fraud/fraud/thirdparty_raw/LOAD0000001F.pa...
1,3e9fc0ce-8b88-4776-a367-a3d1a42b1fc5,2022-05-26 23:42:51.938000-07:00,NaT,a326d7e1-8791-48b4-9d86-bd5b2d0f78aa,"{""kyc"": {""reasonCodes"": [], ""fieldValidations""...","{""dob"": ""1966-02-22"", ""zip"": ""60176"", ""city"": ...",200,anthonybadagliacco7@gmail.com,None,2022-06-22 10:47:48.585521-07:00,,None,2022-06-22 11:01:00.606000-07:00,dms/fraud/fraud/thirdparty_raw/LOAD0000001F.pa...
2,0096ca99-6451-4491-8c11-1e6e16e776bd,2022-05-26 23:57:29.072000-07:00,NaT,a326d7e1-8791-48b4-9d86-bd5b2d0f78aa,"{""kyc"": {""reasonCodes"": [""I919""], ""fieldValida...","{""dob"": ""2003-11-29"", ""zip"": ""11236"", ""city"": ...",200,bcedric26@yahoo.com,None,2022-06-22 10:47:48.585521-07:00,,None,2022-06-22 11:01:00.606000-07:00,dms/fraud/fraud/thirdparty_raw/LOAD0000001F.pa...
3,e2f4202e-3b4d-4d35-83d2-54b07dcfb9cd,2018-11-02 13:58:20.982000-07:00,NaT,6dc16993-ea94-45d8-a0e4-19b95bf1988e,"{""status"": {""reference"": ""1166007"", ""requestId...","{""person"": {""SSN"": {""type"": ""ssn9"", ""number"": ...",200,dflikkema@gmail.com,None,2022-06-22 10:47:48.585521-07:00,,None,2022-06-22 11:30:33.595000-07:00,dms/fraud/fraud/thirdparty_raw/LOAD0000006D.pa...
4,113115fd-3b87-4b4d-9466-4810d9940a46,2018-10-30 14:03:09.395000-07:00,NaT,a326d7e1-8791-48b4-9d86-bd5b2d0f78aa,"{""fraud"": {""scores"": [{""name"": ""Sofi.PL.201711...","{""dob"": ""1980-08-15"", ""zip"": ""92024"", ""city"": ...",200,workman.sean@icloud.com,None,2022-06-22 10:47:48.585521-07:00,,None,2022-06-22 11:30:33.595000-07:00,dms/fraud/fraud/thirdparty_raw/LOAD0000006D.pa...


In [ ]:
profile_daily_transaction_journal